# Import needed libraries

In [1]:
!pip install sacremoses sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 4.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

from tqdm import tqdm

from torch.utils.tensorboard import SummaryWriter

import os

2024-07-15 02:20:04.419735: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-15 02:20:04.419850: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-15 02:20:04.514528: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Device agnostic code

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Set default device
torch.set_default_device(device)
print(f"Default device set to: {device}")

Default device set to: cuda


In [4]:
train_df = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")
train_df.head(5)

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [5]:
train_df.drop(columns=["id", "lang_abv", "language"], inplace=True)

In [6]:
train_df.sample(5)

,premise,hypothesis,label
4571,It is that prospect that may bring Republicans...,Republicans have always defended such inaccura...,1
351,it sure will well good to talk to,Let's talk again soon.,1
10034,Die Endtheorien identifizieren Epochen mit cha...,Die Theorien identifizieren das Alter anhand d...,1
125,"ähm wie denkst du, dass die Zeitung in Colorad...","Glauben Sie, dass die lokalen Zeitungen nicht ...",1
1441,"You see, he said sadly, ""you have no instincts.""",He said that I had no willpower.,2


In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
class CustomDataset(Dataset):
    def __init__(self, df: pd.DataFrame, tokenizer: object, max_length: int):
        self.dataset = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        premise = self.dataset["premise"].iloc[idx]
        hypothesis = self.dataset["hypothesis"].iloc[idx]
        
        token_dict = self.tokenizer.encode_plus(premise, hypothesis, padding=True, truncation=True, max_length=self.max_length, return_tensors="pt")
    
        return {
            "input_ids": token_dict["input_ids"].flatten(),
            "token_type_ids": token_dict["token_type_ids"].flatten(),
            "attention_mask": token_dict["attention_mask"].flatten(),
            "label": torch.tensor(self.dataset["label"].iloc[idx], dtype=torch.long)
        }
    
    @staticmethod
    def collate_fn(batch):
        batch_input_ids = [batch_item["input_ids"] for batch_item in batch]
        batch_type_ids = [batch_item["token_type_ids"] for batch_item in batch]
        batch_attention_masks = [batch_item["attention_mask"] for batch_item in batch]
        
        batch_input_ids = pad_sequence(batch_input_ids, batch_first=True, padding_value=0)
        batch_type_ids  = pad_sequence(batch_type_ids, batch_first=True, padding_value=0) # will get ignored by the attention mask when going through the model; very important
        batch_attention_masks = pad_sequence(batch_attention_masks, batch_first=True, padding_value=0)
        
        return {
            "input_ids": batch_input_ids,
            "token_type_ids": batch_type_ids,
            "attention_mask": batch_attention_masks,
            "labels": torch.stack([batch_item["label"] for batch_item in batch], dim=0)
        }
        

In [9]:
class CustomDatasetSubmission(Dataset):
    def __init__(self, df: pd.DataFrame, tokenizer: object, max_length: int):
        self.dataset = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        premise = self.dataset["premise"].iloc[idx]
        hypothesis = self.dataset["hypothesis"].iloc[idx]
        
        token_dict = self.tokenizer.encode_plus(premise, hypothesis, padding=True, truncation=True, max_length=self.max_length, return_tensors="pt")
    
        return {
            "input_ids": token_dict["input_ids"].flatten(),
            "token_type_ids": token_dict["token_type_ids"].flatten(),
            "attention_mask": token_dict["attention_mask"].flatten(),
            "id": self.dataset["id"].iloc[idx]
        }
    
    @staticmethod
    def collate_fn(batch):
        batch_input_ids = [batch_item["input_ids"] for batch_item in batch]
        batch_type_ids = [batch_item["token_type_ids"] for batch_item in batch]
        batch_attention_masks = [batch_item["attention_mask"] for batch_item in batch]
        
        batch_input_ids = pad_sequence(batch_input_ids, batch_first=True, padding_value=0)
        batch_type_ids  = pad_sequence(batch_type_ids, batch_first=True, padding_value=0) # will get ignored by the attention mask when going through the model; very important
        batch_attention_masks = pad_sequence(batch_attention_masks, batch_first=True, padding_value=0)
        
        return {
            "input_ids": batch_input_ids,
            "token_type_ids": batch_type_ids,
            "attention_mask": batch_attention_masks,
            "ids": [batch_item["id"] for batch_item in batch]
        }
 

In [10]:
max_length = max(max([len(premise) for premise in train_df["premise"]]), max([len(hypothesis) for hypothesis in train_df["hypothesis"]]))
train_split, val_split = train_test_split(train_df, test_size=0.25, shuffle=True)
train_dataset = CustomDataset(df=train_split, tokenizer=tokenizer, max_length=max_length)
val_dataset = CustomDataset(df=val_split, tokenizer=tokenizer, max_length=max_length)

In [11]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=True, generator=torch.Generator(device=device), collate_fn=train_dataset.collate_fn)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=4, shuffle=True, generator=torch.Generator(device=device), collate_fn=val_dataset.collate_fn)

In [12]:
next(iter(val_dataloader))

{'input_ids': tensor([[  101,  1159, 29737, 29723, 18199,  1159, 29731, 18199, 29733, 29155,
           1158, 24824, 29727, 29739, 29733, 29723, 18199,  1169, 29734, 18199,
           1174, 29730,  2012,  2696,  1173, 29735, 29728, 29731, 29723, 29732,
          18199, 29727, 14608, 29728, 29720, 14608, 16177, 29723,  1159, 16177,
          14608,  1170, 29735, 29732, 24824, 16177, 18199, 29726, 29730,  1159,
          29732, 29721, 29730, 29733, 29734, 14608, 29733, 18199, 29730,  1173,
          29734, 29730, 16177,  1170, 29732, 29730, 29726, 14608, 29734, 14608,
          29732, 29726, 29734, 18199, 29726, 29730,  1174, 26789,  1164, 14608,
          29734, 14608, 29727, 29730, 29721, 29730,  1173, 29734, 29730, 29737,
          29739, 16177,  1010,  1155, 29727, 29727, 14608,  1169,  8026, 14804,
           2378,  1155, 29731, 29730, 29736, 14608, 29733, 18199, 29733, 29723,
           1167, 14608,  1159, 29721, 29726, 14608, 29734, 14608, 29727, 29723,
          18199, 29738, 297

In [13]:
bert_model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'google-bert/bert-large-uncased')

/opt/conda/lib/python3.10/site-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [14]:
class Classifier(nn.Module):
    def __init__(self, n_inputs, hidden_size, n_classes, dropout):
        super(Classifier, self).__init__()
        self.linear1 = nn.Linear(in_features=n_inputs, out_features=hidden_size)
        self.linear2 = nn.Linear(in_features=hidden_size, out_features=n_classes)
        self.dropout = nn.Dropout(dropout)

        self.act_fn = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.act_fn(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        return x


In [15]:
class BertClassifier(nn.Module):
    def __init__(self, bert_model, classifier):
        super(BertClassifier, self).__init__()
        self.bert_model = bert_model
        self.classifier = classifier

    def forward(self, x: torch.Tensor, token_type_ids: torch.Tensor, attention_mask: torch.Tensor = None) -> torch.Tensor:
        x = self.bert_model(x, attention_mask=attention_mask, token_type_ids=token_type_ids)
        x = x.last_hidden_state[:, 0, :]
        x = self.classifier(x)
        return x

In [16]:
classifier = Classifier(n_inputs=1024, n_classes=3, hidden_size=768, dropout=0.35)
model = BertClassifier(bert_model=bert_model, classifier=classifier)
bert_model.config.hidden_dropout_prob = 0.35
bert_model.config

BertConfig {
  "_name_or_path": "google-bert/bert-large-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.35,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.42.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [17]:
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()

In [18]:
def train_model(model, dataloader, val_dataloader, loss_fn, optimizer, scheduler, epochs):
    writer = SummaryWriter()
    writer.add_graph(model, (torch.zeros(size=(32, 32), dtype=torch.long), torch.zeros(size=(32, 32), dtype=torch.long)))
    for epoch in range(epochs):
        model.eval()
        total_inputs = 0
        total_correct = 0

        with torch.inference_mode():
            for batch in tqdm(val_dataloader):
                logits = model(batch["input_ids"], attention_mask=batch["attention_mask"], token_type_ids=batch["token_type_ids"])
                percents = torch.softmax(logits, dim=1)
                preds = torch.argmax(percents, dim=1)

                total_correct += (preds==batch["labels"]).sum().item()
                total_inputs += batch["labels"].view(-1).shape[0]

            print(f"{total_correct} out of {total_inputs}")
            print(f"acc of {total_correct/total_inputs*100}%")

            writer.add_scalar("Val acc", total_correct/total_inputs*100, epoch)

        model.train()
        for batch_idx, batch in tqdm(enumerate(dataloader)):
            logits = model(batch["input_ids"], attention_mask=batch["attention_mask"], token_type_ids=batch["token_type_ids"])
            loss = loss_fn(logits, batch["labels"])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (batch_idx+1) % 40 == 0:
                print(f"loss for batch {batch_idx+1} --> {loss} at epoch {epoch}")

            writer.add_scalar("Loss", loss, batch_idx)

        scheduler.step()

    writer.close()

In [19]:
scheduler = StepLR(gamma=0.5, step_size=1, optimizer=optimizer)
train_model(model=model, dataloader=train_dataloader, val_dataloader=val_dataloader, loss_fn=loss_fn, optimizer=optimizer, scheduler=scheduler, epochs=3)

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:4565: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
100%|██████████| 758/758 [01:08<00:00, 11.14it/s]


1018 out of 3030
acc of 33.5973597359736%


40it [00:29,  1.17it/s]

loss for batch 40 --> 0.9956830143928528 at epoch 0


80it [00:56,  1.48it/s]

...

1080it [12:59,  1.12it/s]

loss for batch 1080 --> 0.4389224648475647 at epoch 2


1120it [13:27,  1.72it/s]

loss for batch 1120 --> 0.1273920238018036 at epoch 2


1137it [13:39,  1.39it/s]


In [20]:
submission_csv = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/test.csv")
submission_csv.drop(columns=["lang_abv", "language"])

,id,premise,hypothesis
0,c6d58c3f69,بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...,"کیسی کے لئے کوئی یادگار نہیں ہوگا, کولمین ہائی..."
1,cefcc82292,هذا هو ما تم نصحنا به.,عندما يتم إخبارهم بما يجب عليهم فعله ، فشلت ال...
2,e98005252c,et cela est en grande partie dû au fait que le...,Les mères se droguent.
3,58518c10ba,与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp,IMA与其他组织合作，因为它们都依靠共享资金。
4,c32b0d16df,Она все еще была там.,"Мы думали, что она ушла, однако, она осталась."
...,...,...,...
5190,5f90dd59b0,نیند نے وعدہ کیا کہ موٹل نے سوال میں تحقیق کی.,نیمیتھ کو موٹل کی تفتیش کے لئے معاوضہ دیا جارہ...
5191,f357a04e86,The rock has a soft texture and can be bough...,The rock is harder than most types of rock.
5192,1f0ea92118,她目前的存在，并考虑到他与沃佛斯顿争执的本质，那是尴尬的。,她在与Wolverstone的打斗结束后才在场的事实被看作是很尴尬的。
5193,0407b48afb,isn't it i can remember i've only been here ei...,I could see downtown Dallas from where I lived...


In [21]:
submission_dataset = CustomDatasetSubmission(df=submission_csv.drop(columns=["lang_abv", "language"]), max_length=max_length, tokenizer=tokenizer)

In [22]:
submission_dataloder = DataLoader(dataset=submission_dataset, batch_size=2, shuffle=False, generator=torch.Generator(device=device), collate_fn=submission_dataset.collate_fn)

In [23]:
final_output = {
    "id": [],
    "prediction": []
}

model.eval()
with torch.inference_mode():
    for batch in tqdm(submission_dataloder):
        logits = model(batch["input_ids"], attention_mask=batch["attention_mask"], token_type_ids=batch["token_type_ids"])
        percents = torch.softmax(logits, dim=1)
        preds = torch.argmax(percents, dim=1)
        for idx, prediction in zip(batch["ids"], preds.view(-1).tolist()):
            final_output["id"].append(idx)
            final_output["prediction"].append(prediction)


100%|██████████| 2598/2598 [01:54<00:00, 22.62it/s]


In [24]:
final_output_df = pd.DataFrame(final_output)
final_output_df.to_csv("./submission.csv", index=False)

In [25]:
final_output_df

,id,prediction
0,c6d58c3f69,1
1,cefcc82292,1
2,e98005252c,0
3,58518c10ba,1
4,c32b0d16df,2
...,...,...
5190,5f90dd59b0,1
5191,f357a04e86,1
5192,1f0ea92118,1
5193,0407b48afb,0
